In [121]:
#import all libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [122]:
#load data 
world_cup = pd.read_csv('./World Cup 2023 Dataset.csv')
results = pd.read_csv('./results.csv')

In [123]:
world_cup.head()

,Team,Group,Previous \nappearances,Previous \r\ntitles,Previous\r\n finals,Previous\r\n semifinals,Current \r rank
0,England,A,12,1,4,6,7
1,South Africa,A,7,0,0,5,2
2,Netherlands,A,5,0,0,0,10
3,Pakistan,A,12,1,2,6,5
4,New Zealand,A,12,0,2,9,4


In [124]:
results.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,"Jan 12, 2011",South Africa,India,South Africa,135 runs,Durban
1,"Jan 15, 2011",South Africa,India,India,1 run,Johannesburg
2,"Jan 16, 2011",Australia,England,Australia,6 wickets,Melbourne
3,"Jan 18, 2011",South Africa,India,India,2 wickets,Cape Town
4,"Jan 21, 2011",Australia,England,Australia,46 runs,Hobart


In [125]:
df = results[(results['Team_1'] == 'India') | (results['Team_2'] == 'India')]
india = df.iloc[:]
india.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,"Jan 12, 2011",South Africa,India,South Africa,135 runs,Durban
1,"Jan 15, 2011",South Africa,India,India,1 run,Johannesburg
3,"Jan 18, 2011",South Africa,India,India,2 wickets,Cape Town
5,"Jan 21, 2011",South Africa,India,South Africa,48 runs,Gqeberha
8,"Jan 23, 2011",South Africa,India,South Africa,33 runs,Centurion


In [126]:
#narrowing to team patcipating in the world cup
worldcup_teams = ['England', ' South Africa', 'Netherlands', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India']
df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()
df_teams.count()

date      1870
Team_1    1870
Team_2    1870
Winner    1870
Margin    1870
Ground    1870
dtype: int64

In [127]:
df_teams.head()

,date,Team_1,Team_2,Winner,Margin,Ground
2,"Jan 16, 2011",Australia,England,Australia,6 wickets,Melbourne
4,"Jan 21, 2011",Australia,England,Australia,46 runs,Hobart
6,"Jan 22, 2011",New Zealand,Pakistan,New Zealand,9 wickets,Wellington
7,"Jan 23, 2011",Australia,England,Australia,4 wickets,Sydney
9,"Jan 26, 2011",New Zealand,Pakistan,no result,-,Queenstown


In [128]:
#dropping columns that wll not affect match outcomes
df_teams_2010 = df_teams.drop(['date','Margin', 'Ground'], axis=1)
df_teams_2010.head()

,Team_1,Team_2,Winner
2,Australia,England,Australia
4,Australia,England,Australia
6,New Zealand,Pakistan,New Zealand
7,Australia,England,Australia
9,New Zealand,Pakistan,no result


In [129]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2010 = df_teams_2010.reset_index(drop=True)
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_1,'winning_team']=1
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_2, 'winning_team']=2
df_teams_2010 = df_teams_2010.drop(['winning_team'], axis=1)

df_teams_2010.head()

,Team_1,Team_2,Winner
0,Australia,England,Australia
1,Australia,England,Australia
2,New Zealand,Pakistan,New Zealand
3,Australia,England,Australia
4,New Zealand,Pakistan,no result


In [130]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_2010, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [131]:
final.head()

,Winner,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_Canada,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Kenya,...,Team_2_New Zealand,Team_2_Oman,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_U.S.A.,Team_2_West Indies,Team_2_Zimbabwe
0,Australia,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Australia,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,New Zealand,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Australia,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,no result,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [132]:

rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.704
Test set accuracy:  0.599


In [133]:
#adding ICC rankings
#the team which is positioned higher on the ICC Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

# Loading new datasets
ranking = pd.read_csv('./icc_rankings.csv') 
fixtures = pd.read_csv('./fixtures.csv')

# List for storing the group stage games
pred_set = []

In [134]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
40,1,5.0,7.0,"Nov 9, 2023",Bengaluru,New Zealand,Sri Lanka,Group A,NaN
41,1,9.0,3.0,"Nov 10, 2023",Ahmedabad,Afghanistan,South Africa,Group A,NaN
42,1,2.0,8.0,"Nov 11, 2023",Pune,Australia,Bangladesh,Group A,NaN
43,1,6.0,4.0,"Nov 11, 2023",Eden Gardens,England,Pakistan,Group A,NaN
44,1,1.0,NaN,"Nov 12, 2023",Bengaluru,India,Netherlands,Group A,NaN


In [135]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,New Zealand,England,None
1,Pakistan,Netherlands,None
2,Bangladesh,Afghanistan,None
3,South Africa,Sri Lanka,None
4,India,Australia,None


In [136]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_Canada,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Kenya,Team_1_Nepal,...,Team_2_New Zealand,Team_2_Oman,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_U.S.A.,Team_2_West Indies,Team_2_Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [137]:
#group matches 
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

England and New Zealand
Winner: New Zealand

Netherlands and Pakistan
Winner: Pakistan

Afghanistan and Bangladesh
Winner: Bangladesh

Sri Lanka and South Africa
Winner: South Africa

Australia and India
Winner: India

Netherlands and New Zealand
Winner: New Zealand

Bangladesh and England
Winner: England

Sri Lanka and Pakistan
Winner: Pakistan

Afghanistan and India
Winner: India

South Africa and Australia
Winner: Australia

Bangladesh and New Zealand
Winner: New Zealand

Pakistan and India
Winner: India

Afghanistan and England
Winner: England

Sri Lanka and Australia
Winner: Australia

Netherlands and South Africa
Winner: South Africa

Afghanistan and New Zealand
Winner: New Zealand

Bangladesh and India
Winner: India

Pakistan and Australia
Winner: Australia

Netherlands and Sri Lanka
Winner: Sri Lanka

England and South Africa
Winner: South Africa

New Zealand and India
Winner: India

Afghanistan and Pakistan
Winner: Pakistan

Bangladesh and South Africa
Winner: South Africa

Au

In [138]:
# List of tuples before 
semi = [('New Zealand', 'India'),
            ('Australia', 'South Africa')]

In [139]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [140]:
clean_and_predict(semi, ranking, final, rf)

New Zealand and India
Winner: India

South Africa and Australia
Winner: Australia



In [141]:
# Finals
finals = [('India', 'Australia')]

In [142]:
clean_and_predict(finals, ranking, final, rf)

Australia and India
Winner: India

